# Tensor Methods Homework 6

In [1]:
using Base.Iterators, LinearAlgebra, Plots
import Base: *, /

In [2]:
# strong Kronecker product for 3rd order tensors
function strongKron(A::AbstractArray{<:Number, 3}, B::AbstractArray{<:Number, 3})
    m = size(A)
    n = size(B)
    @assert m[3] == n[1] "Shared rank index does not match: got $(m[3]) and $(n[1]) respectively."

    C = reshape(A, m[1]*m[2], m[3]) * reshape(B, n[1], n[2]*n[3])
    return reshape(C, m[1], m[2]*n[2], n[3])
end

⨝(A::AbstractArray{<:Number, 3}, B::AbstractArray{<:Number, 3}) = strongKron(A, B)

;

In [3]:
# MPS-TT type as a list of 3D-tensor factors
MPSTT{T<:Number} = AbstractVector{<:AbstractArray{T,3}}

# returns number of dimensions of an MPS-TT
dims(U::MPSTT) = length(U)

# returns the mode dimensions of an MPS-TT
function shape(U::MPSTT)
    n = [size(factor, 2) for factor in U]
    return Tuple(n)
end

# returns the ranks of an MPS-TT
function ranks(U::MPSTT)
    r = [size(factor, 1) for factor in U]
    r = r[2:end] # remove dummy dimension
    return Tuple(r)
end

# construct tensor from MPS-TT factors
function evalMPSTT(U::MPSTT)
    n = shape(U)
    A = reduce(strongKron, U, init=[])
    return reshape(A, n)
end

;

### 1. Implementing the MPS-TT truncation.

Implement the MPS-TT rounding (rank truncation) algorithm. Given
- a vector $u \in \mathbb{R}^{n_1 \dots n_d}$ (where $d \in \mathbb{N}$ and $n_1, \dots, n_d \in \mathbb{N}$) in the form of an MPS-TT factorization $u = U_1 \ {\scriptsize\bowtie} \ \dots \ {\scriptsize\bowtie} \ U_d$ with
ranks $p_1, \dots, p_{d−1} \in \mathbb{N}$ and 
- target ranks $r_1, \dots, r_{d−1} \in \mathbb{N}$,

the implementation should produce an MPS-TT approximation $v = V_1 \ {\scriptsize\bowtie} \ \dots \ {\scriptsize\bowtie} \ V_d$ to $u$ of quasi-optimal accuracy in the Frobenius norm and of ranks not exceeding $r_1, \dots, r_{d−1}$.

The output should include
- the MPS-TT decomposition produced by the algorithm,
- the vectors of singular values of the matrices that are explicitly approximated within the algorithm (one vector of singular values per step) and
- the Frobenius norms of the errors of the mentioned low-rank matrix approximation (one scalar per step).

In [4]:
function roundMPSTT(U::MPSTT, p::NTuple{N,<:Integer}) where N

    # initialization
    d = dims(U)
    n = shape(U)
    r = (1, ranks(U)..., 1)
    p = (1, p..., 1)
    V = []
    singular_vals = Vector{Vector{Float64}}(undef, d-1)
    norms = Vector{Float64}(undef, d-1)

    # check input
    @assert N == d-1 "Expected $(d-1) rank(s), got $N."
    for k ∈ 1:d-1
        upper = min(p[k] * n[k], prod(n[k+1:end]))
        upper = min(upper, r[k+1])
        @assert 0 < p[k+1] <= upper "Rank at index $k is out of bounds: expected in range 1:$upper, got $(p[k+1])."
    end

    # left orthogonalization
    R = ones(1, 1, 1)
    for k ∈ d:-1:2
        UR = U[k] ⨝ R

        UR = permutedims(UR, [3,2,1]) # permute dims for right QR
        UR = reshape(UR, r[k+1]*n[k], r[k])

        Q, R = qr(UR)

        Q = reshape(Matrix(Q), r[k+1], n[k], r[k])
        R = reshape(R, r[k], 1, r[k])

        Q = permutedims(Q, [3,2,1])
        R = permutedims(R, [3,2,1])

        push!(V, Q)
    end

    # compute first factor
    push!(V, U[1] ⨝ R)
    reverse!(V)

    # right approximation
    S = ones(1, 1, 1)
    for k ∈ 1:d-1
        SV = S ⨝ V[k]
        SV = reshape(SV, p[k]*n[k], r[k+1])

        F = svd(SV)
        Vk = F.U[:,1:p[k+1]]
        S = F.Vt[1:p[k+1],:] .* F.S[1:p[k+1]]

        singular_vals[k] = F.S
        norms[k] = norm(SV - Vk * S)

        Vk = reshape(Vk, p[k], n[k], p[k+1])
        S = reshape(S, p[k+1], 1, r[k+1])

        V[k] = Vk
    end

    # compute last factor
    V[d] = S ⨝ V[d]

    # bring all factors to the same type
    T = Base.promote_eltype(V...)
    V = [convert(Array{T}, factor) for factor ∈ V]

    return V, singular_vals, norms
end

roundMPSTT(U::MPSTT, r::AbstractVector{<:Integer}) = round(U, tuple(r...))

;

### 2. Adding, scaling and multiplying in the TT-MPS representation.

Implement functions that, for $d \in \mathbb{N}$ and $m_1, \dots, m_d, n_1, \dots, n_d \in \mathbb{N}$, compute exact MPS-TT
factorizations $w = W_1 \ {\scriptsize\bowtie} \ \dots \ {\scriptsize\bowtie} \ W_d$

a) of the linear combination $w = \alpha u + \beta v ∈ \mathbb{R}^{n_1 \dots n_d}$, with given coefficients $\alpha, \beta \in \mathbb{R}$, of two vectors $u, v \in \mathbb{R}^{n_1 \dots n_d}$ given in the form of MPS-TT decompositions $u = U_1 \ {\scriptsize\bowtie} \ \dots \ {\scriptsize\bowtie} \ U_d$ and $v = V_1 \ {\scriptsize\bowtie} \ \dots \ {\scriptsize\bowtie} \ V_d$

In [5]:
function scaleMPSTT(α::Number, A::MPSTT)
    V = copy(A)
    V[1] = α * V[1]
    return V
end

function addMPSTT(A::MPSTT, B::MPSTT)
    d = dims(A)
    @assert d == dims(B) "Number of dimensions do not match: Got $d and $(dims(B))."
    @assert shape(A) == shape(B) "Mode dimensions do not match: Got $(shape(A)) and $(shape(B))."

    first_factor = cat(A[1], B[1], dims=3)
    center_factors = [cat(A[k], B[k], dims=(1,3)) for k ∈ 2:d-1]
    last_factor = cat(A[d], B[d], dims=1)

    return [first_factor, center_factors..., last_factor]
end


*(α::Number, A::MPSTT) = scaleMPSTT(α, A)
*(A::MPSTT, α::Number) = scaleMPSTT(α, A)
/(U::MPSTT, α::Number) = scaleMPSTT(1/α, A)
⊕(A::MPSTT, B::MPSTT) = addMPSTT(A, B)
;

b) of the entrywise product $w = u \odot v \in \mathbb{R}^{n_1 \dots n_d}$, of two vectors $u, v \in \mathbb{R}^{n_1 \dots n_d}$ given in the form of MPS-TT decompositions $u = U_1 \ {\scriptsize\bowtie} \ \dots \ {\scriptsize\bowtie} \ U_d$ and $v = V_1 \ {\scriptsize\bowtie} \ \dots \ {\scriptsize\bowtie} \ V_d$;

In [6]:
function entrymulFactor(Ak::AbstractArray{TA,3}, Bk::AbstractArray{TB,3}) where {TA<:Number, TB<:Number}
    n = size(Ak, 2)
    slices = [kron(Ak[:,i,:], Bk[:,i,:]) for i ∈ 1:n]
    Ck = cat(slices..., dims=3)
    return permutedims(Ck, [1,3,2])
end

function entrymulMPSTT(A::MPSTT, B::MPSTT)
    d = dims(A)
    n = shape(A)
    @assert d == dims(B) "Number of dimensions do not match: Got $d and $(dims(B))."
    @assert n == shape(B) "Mode dimensions do not match: Got $n and $(shape(B))."

    return [entrymulFactor(A[k], B[k]) for k ∈ 1:d]
end

⊙(A::MPSTT, B::MPSTT) = entrymulMPSTT(A, B)

;

c) of the product $w = A u \in \mathbb{R}^{m_1 \dots m_d}$ of a given matrix $A \in \mathbb{R}^{m_1 \dots m_d \times n_1 \dots n_d}$ and a given vector $u \in \mathbb{R}^{n_1 \dots n_d}$ given in the form of MPS-TT decompositions $A = A_1 \ {\scriptsize\bowtie} \ \dots \ {\scriptsize\bowtie} \ A_d$ and $u = U_1 \ {\scriptsize\bowtie} \ \dots \ {\scriptsize\bowtie} \ U_d$.

In [16]:
function matvecmulFactor(Ak::AbstractArray{TA,3}, uk::AbstractArray{Tu,3}) where {TA<:Number, Tu<:Number}
    mn = size(Ak, 2)
    n = size(uk, 2)
    m = Int(mn / n)
    Bk = reshape(Ak, size(Ak,1), m, n, size(Ak,3))
    slices = [sum([kron(Ak[:,i,j,:], uk[:,j,:]) for j ∈ 1:n]) for i ∈ 1:m]
    Ck = cat(slices..., dims=3)
    return permutedims(Ck, [1,3,2])
end

function matvecmulMPSTT(A::MPSTT, u::MPSTT)
    d = dims(A)
    @assert d == dims(u) "Number of dimensions do not match: Got $d and $(dims(u))."
    n = shape(u)
    m = shape(A) ./ n
    @assert all(isinteger.(m)) "Mode dimensions do not match: Got $m and $n."

    return [matvecmulFactor(A[k], u[k]) for k ∈ 1:d]
end

⊗(A::MPSTT, u::MPSTT) = matvecmulMPSTT(A, u)

;

### 3. Testing the MPS-TT arithmetic.

For $n = 51$, let us consider the grid of $t_i = 2 \frac{i − 1}{n − 1} − 1$ with $i = 1, \dots, n$, the tensors $X, Y \in \mathbb{R}^{n \times n \times n \times n}$ given by
$$ X_{i_1, i_2, i_3, i_4} = T_p \left( \sum_{k=1}^4 \frac{t_{i_k}}{k} \right) $$
and
$$ Y_{i_1, i_2, i_3, i_4} = T_q \left( \sum_{k=1}^4 \frac{t_{i_k}}{k} \right) $$
where $p, q \in \mathbb{N}_0$ and $T_r$ with $r \in \mathbb{N}_0$ is the Chebyshev polynomial of the first kind of degree $r$, and their vectorizations $x = \mathrm{vec}(X)$ and $y = \mathrm{vec}(Y)$.

Consider the sum $s = x + y$ and the entrywise product $z = x \odot y$.

In [44]:
# Chebyshev polynomial of first kind
function T(p::Integer)
    function Tx(x)
        if abs(x) <= 1
            return cos(p * acos(x))
        elseif x > 1
            return cosh(p * acosh(x))
        else # x < 1
            return (-1)^p * cosh(p * acosh(-x))
        end
    end
    return Tx
end

function getTensor(p::Integer, d::Integer, grid::Vector{<:Number})
    f = T(p)
    n = length(grid)

    repeated_grid = ntuple(_ -> grid, d)
    meshgrid = map(collect, product(repeated_grid...))

    return [f(sum(x ./ (1:d))) for x ∈ meshgrid]
end

;

In [42]:
d = 4
n = 51
t = collect(2 * ((1:n) .- 1) / (n-1) .- 1)
;

a) Inspect the decay of the singular values for all the MPS-TT unfoldings of $x, y, s, z$ for several moderate values of $p$ and $q$ (for example, 3 and 4, 5 and 7) For each of these vectors, how does the maximum of the MPS-TT ranks depend on $p$ and $q$? Are the bounds derived for such vectors in the lectures sharp for the examples you have considered?

In [45]:
ps = (3, 5)
qs = (4, 7)

for (p, q) ∈ zip(ps, qs)
    X = getTensor(p, d, t)
    Y = getTensor(q, d, t)
    x = vec(X)
    y = vec(Y)
    s = x + y
    z = x .* y
end

b) Let $p = 5$ and $q = 7$. Compute exact (up to machine precision) MPS-TT representations of $x$ and $y$ using an implementation of the TT-SVD (Schmidt decomposition) algorithm (see assignment 4). Use your implementation from problem 2 to directly assemble MPS-TT representations of $s$ and $z$ without forming these vectors entrywise. Use your implementation of the rank truncation algorithm from problem 1 to compute exact (up to machine precision) MPS-TT representations of $s$ and $z$. Check that these decompositions are exact up to machine precision by forming the vectors entrywise (as $x + y$ and $x \odot y$).

In [14]:
P = 5
q = 7

7